In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
tf.VERSION

'1.15.0'

In [4]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.advanced_activations import LeakyReLU
from functools import partial
from keras.layers.merge import _Merge
import keras.backend as K
import numpy as np

img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
optimizer = Adam(0.0002, 0.5)
num_classes = 10

def Encoder():
  model = Sequential()
  model.add(Flatten(input_shape = img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(latent_dim))
  model.summary()

  img = Input(shape=img_shape)
  z = model(img)
  return Model(inputs = img, outputs = z)
        
def Generator():
  model = Sequential()
  model.add(Dense(512, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))
  model.summary()

  z = Input(shape=(latent_dim,))
  gen_img = model(z)
  return Model(inputs = z, outputs = gen_img)

def Discriminator():
  z = Input(shape=(latent_dim, ))
  img = Input(shape=img_shape)
  d_in = concatenate([z, Flatten()(img)])

  model = Dense(1024)(d_in)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  model = Dense(1024)(model)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  model = Dense(1024)(model)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  validity = Dense(1, activation="sigmoid")(model)

  return Model(inputs = [z, img], outputs = validity)


# Build and compile the discriminator
D = Discriminator()
D.compile(loss=['binary_crossentropy'], optimizer=optimizer, metrics=['accuracy'])

# Build the generator
G = Generator()

# Build the encoder
E = Encoder()

# The part of the bigan that trains the discriminator and encoder
D.trainable = False

# Generate image from sampled noise
z = Input(shape=(latent_dim, ))
img_ = G(z)

# Encode image
img = Input(shape = img_shape)
z_ = E(img)

# Latent -> img is fake, and img -> latent is valid
fake = D([z, img_])
valid = D([z_, img])

# Set up and compile the combined model
# Trains generator to fool the discriminator
Bigan_G = Model([z, img], [fake, valid])
Bigan_G.compile(loss=['binary_crossentropy', 'binary_crossentropy'], optimizer=optimizer)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 512)               0         
_____________________________________

In [7]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
import os
import numpy as np

os.chdir('/content/drive/My Drive/GitHub Repositories/Generative Models Papers with Implementation in Keras')
baseDir = './BiGAN-Bidirectional Generative Adversarial Network'
outputDir = os.path.join(baseDir, 'outputs')
if not os.path.exists(outputDir):
  os.makedirs(outputDir)
  print('Output Directory Created to save Results')

epochs=40000
batch_size=32
sample_interval=50

def save_imgs(epoch):
  r, c = 5, 5
  z = np.random.normal(size=(25, latent_dim))
  gen_imgs = G.predict(z)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig(outputDir+'/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()
  plt.close()

# Load the dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):
  # ---------------------
  #  Train Discriminator
  # ---------------------

  # Sample noise and generate img
  z = np.random.normal(size=(batch_size, latent_dim))
  imgs_ = G.predict(z)

  # Select a random batch of images and encode
  idx = np.random.randint(0, X_train.shape[0], batch_size)
  imgs = X_train[idx]
  z_ = E.predict(imgs)

  # Train the discriminator (img -> z is valid, z -> img is fake)
  d_loss_real = D.train_on_batch([z_, imgs], valid)
  d_loss_fake = D.train_on_batch([z, imgs_], fake)
  d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

  # ---------------------
  #  Train Generator
  # ---------------------

  # Train the generator (z -> img is valid and img -> z is is invalid)
  g_loss = Bigan_G.train_on_batch([z, imgs], [valid, fake])

  # Plot the progress
  print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

  # If at save interval => save generated image samples
  if epoch % sample_interval == 0:
      save_imgs(epoch)

    




Output hidden; open in https://colab.research.google.com to view.

In [0]:
import imageio
import glob
images = []
for file_name in os.listdir(outputDir):
    if file_name.endswith('.png'):
        file_path = os.path.join(outputDir, file_name)
        images.append(imageio.imread(file_path))
imageio.mimsave(os.path.join(baseDir, 'BiGAN-training-outputs.gif'), images, fps = 5)